# CAI Lab Session 2: Intro to ElasticSearch

In this session you will learn:

- a few basics of the `ElasticSearch` database
- how to index a set of documents and how to ask simple queries about these documents
- how to do this from `Python`
- based on the previous, you will compute the boolean and tf-idf matrix for the toy corpus used in class

## 1. ElasticSearch

[ElasticSearch](https://www.elastic.co/) is a _NoSQL/document_ database with the capability of indexing and searching text documents. As a rough analogue, we can use the following table for the equivalence between ElasticSearch and a more classical relational database:

| Relational DB | ElasticSearch |
|---|---|
| Database | Index |
| Table | Type |
| Row / record | Document |
| Column | Field |

An index can be thought of as an optimized collection of documents and each document is a collection of fields, which are the key-value pairs that contain your data.

`ElasticSearch` is a pretty big beast with many options. Luckily, there is much documentation, a few useful links are:

- Here is the [full documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)
- Intros you may want to have a look at: 
    - https://medium.com/expedia-group-tech/getting-started-with-elastic-search-6af62d7df8dd
    - http://joelabrahamsson.com/elasticsearch-101
- You found another one that you liked? Let us know. 

## 2. Running ElasticSearch

First you will need to install `ElasticSearch` following instructions in their [documentation](https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html).

This database runs as a web service in a machine and can be accessed using a REST web API; however we will interact with the database through its python libraries `elasticsearch-py` and `elasticsearch-dsl`, so you will need to install these as well.  You can run `ElasticSearch` by typing from the command-line prompt:

```
$ <path_to_elasticsearch_bin>/elasticsearch &
```



After a few seconds (and a lot of logging) the database will be up and running; you may need to hit return for the prompt to show up. To test whether `ElasticSearch` is working execute the code in the cell below

In [1]:
from pprint import pprint

In [2]:
import requests

try:
    resp = requests.get('http://localhost:9200/')
    pprint(resp.content)
    
except Exception:
    print('elasticsearch is not running')

elasticsearch is not running


If `ElasticSearch` is working you will see an answer from the server; otherwise you will see a message indicating that it is not running. You can try also throwing the URL http://localhost:9200 to your browser; you should get a similar answer.

**In version 8 they introduced enhanced security, which may give you trouble when executing the code here, to deal with this you can either install an earlier version (7 or older) or turn off security settings in their `config/elasticsearch.yml` config file (just set to _false_ everything concerning the security options).** Since we are using the database in offline, local mode this should not be a problem.

Also, you should run this script locally in your machine, if you use Google Collab or similar this is not going to work because elasticsearch should be running on the machine where the script is being executed.

## 3. Indexing and querying

`ElasticSearch` is a database that allows storing documents (tables do not need a predefined schema as in relational databases). Text in these documents can be processed so the queries extend beyond exact matches allowing complex queries, fuzzy matching and ranking documents respect to the actual match. 

These kinds of databases are behind search engines like Google Search or Bing.

There are different ways of operating with ElasticSearch. It is deployed esentially as a web service with a REST API, so it can be accessed basically from any language with a library for operating with HTTP servers.

We are going to use two python libraries for programming on top of ElasticSearch: `elasticsearch` and `elasticsearch-dsl`. Both provide access to ElasticSearch functionalities hiding and making more programming-friendly the interactions, the second one is more convenient for configurating and searching. Make sure both python libraries are installed to proceed with this session.

In [18]:
!pip3 install elasticsearch --user
!pip3 install elasticsearch-dsl --user

We are only going to see the essential elements for developing the session but feel free to learn more.

To interact with ElasticSearch with need a client object of type `Elasticsearch`.

In [19]:
from elasticsearch import Elasticsearch

client = Elasticsearch("http://localhost:9200")

With this client you have a connection for operating with Elastic search. Now we will create an index. There are index operations in each library, but the one in `elasticseach-dsl` is simpler to use.

In [20]:
from elasticsearch_dsl import Index

index = Index('test', using=client)

First we will need some text to index, for testing purposes we are going to use the python library `loremipsum`. We will need to install it first if it is not installed already, uncomment the code in next cell if you need to install the library

In [21]:
!pip3 install lorem --user  # Restart the kernel if you are not able to import the library in the next cell

Now we create some random paragraphs

In [22]:
import lorem

texts = [lorem.paragraph() for _ in range(10)]
print(len(texts))
print(texts[0])

10
Quisquam ipsum ut sed. Est numquam adipisci quaerat modi. Quiquia quaerat est dolore modi amet. Non voluptatem eius non. Ipsum est non labore. Quisquam quiquia sit magnam amet neque porro. Etincidunt sit etincidunt sit sed ut. Aliquam tempora sit quaerat tempora quisquam magnam velit. Quiquia voluptatem est dolorem. Velit ipsum etincidunt amet numquam dolore.


Now we can index the paragraphs in ElasticSearch using the `index` method. The document is passed as a python dictionary with the `body` parameter. The keys of the dictionary will be the fields of the document, in this case we well have only one (`text`) -- here, we use this tag but could use anything we wanted to.

In [23]:
for t in texts:
    client.index(index='test', document={'text': t})
    print(f'Indexing new text: {t[:70]} ...')

Indexing new text: Quisquam ipsum ut sed. Est numquam adipisci quaerat modi. Quiquia quae ...
Indexing new text: Dolore tempora eius aliquam consectetur quaerat ipsum. Eius numquam ei ...
Indexing new text: Ut quaerat quiquia quaerat numquam. Velit dolorem dolore velit tempora ...
Indexing new text: Magnam consectetur quisquam magnam aliquam. Quiquia numquam labore est ...
Indexing new text: Aliquam magnam ipsum amet consectetur. Dolore velit aliquam amet dolor ...
Indexing new text: Aliquam dolorem etincidunt porro eius. Consectetur dolor ipsum ut ut q ...
Indexing new text: Aliquam quisquam consectetur non amet voluptatem voluptatem neque. Ame ...
Indexing new text: Est dolor neque voluptatem velit quiquia. Dolorem ut adipisci quisquam ...
Indexing new text: Eius dolor non quisquam dolor labore. Quisquam dolore dolore aliquam a ...
Indexing new text: Magnam amet adipisci est dolore aliquam. Neque tempora labore amet por ...


In case we want to get all docs in the index, we can do the following:

In [24]:
# get all docs in index 'test'
resp = client.search(index="test", query={"match_all": {}})

# print them
print(f"Got {resp['hits']['total']['value']} hits:")
for hit in resp['hits']['hits']:
    pprint(hit["_source"])

Got 8 hits:
{'text': 'Quisquam ipsum ut sed. Est numquam adipisci quaerat modi. Quiquia '
         'quaerat est dolore modi amet. Non voluptatem eius non. Ipsum est non '
         'labore. Quisquam quiquia sit magnam amet neque porro. Etincidunt sit '
         'etincidunt sit sed ut. Aliquam tempora sit quaerat tempora quisquam '
         'magnam velit. Quiquia voluptatem est dolorem. Velit ipsum etincidunt '
         'amet numquam dolore.'}
{'text': 'Dolore tempora eius aliquam consectetur quaerat ipsum. Eius numquam '
         'eius labore ipsum non quisquam. Consectetur sit amet adipisci neque '
         'sed. Aliquam etincidunt velit amet magnam non. Velit aliquam porro '
         'quaerat ipsum numquam. Consectetur quaerat aliquam consectetur '
         'quiquia dolorem porro.'}
{'text': 'Ut quaerat quiquia quaerat numquam. Velit dolorem dolore velit '
         'tempora voluptatem eius. Est etincidunt labore magnam. Dolore '
         'adipisci aliquam numquam quiquia tempora. Ut d

We can also search for documents that contain a given keyword:

In [25]:
from elasticsearch_dsl import Search

# the following search query specifies the field where we want to search
s_obj = Search(using=client, index='test')
sq = s_obj.query('match', text='dolor')
resp = sq.execute()

print(f'Found {len(resp)} matches.')

for hit in resp:
    print(f'\nID: {hit.meta.id}\nText: {hit.text}')

Found 5 matches.

ID: h_fkk4oBvuzMbP3JNVXJ
Text: Aliquam magnam ipsum amet consectetur. Dolore velit aliquam amet dolor dolorem labore. Consectetur consectetur sit aliquam numquam aliquam velit. Dolore sed ut eius voluptatem numquam adipisci. Porro dolor quaerat dolor dolor. Voluptatem voluptatem aliquam dolore. Velit quaerat voluptatem quisquam magnam. Quaerat ipsum eius est.

ID: ivfkk4oBvuzMbP3JN1W-
Text: Est dolor neque voluptatem velit quiquia. Dolorem ut adipisci quisquam ut. Aliquam dolor velit sit dolor magnam. Non adipisci aliquam non. Eius sed quaerat voluptatem. Ipsum sit magnam dolor labore quisquam sed aliquam. Ipsum consectetur quaerat magnam tempora dolore amet. Est consectetur labore porro. Ut consectetur adipisci labore sit.

ID: hvfkk4oBvuzMbP3JNVUr
Text: Magnam consectetur quisquam magnam aliquam. Quiquia numquam labore est est dolore dolorem. Amet dolorem etincidunt adipisci modi voluptatem quiquia. Ipsum voluptatem consectetur tempora velit. Labore quiquia dolore q

## 4. Counting words and docs

`Elastic search` helps us to obtain the counts of words in each document. For example, the following code obtains the counts of words of a whole index by adding the counts of words obtained from each document through the functionality of `termvectors`. This function also allows us to get _document counts_ for computing tf-idf weights, by setting the `term_statistics` option to `True`.

In [26]:
from elasticsearch.helpers import scan
from collections import Counter

# Search for all the documents and query the list of (word, frequency) of each one
# Totals are accumulated using a Counter
word_counts = Counter()
sc = scan(client, index='test', query={"query" : {"match_all": {}}})
for s in sc:
    doc_counts = Counter()   # I place the counter here so that it is overwritten each time, since doc_freq is constant for every doc
    tv = client.termvectors(index='test', id=s['_id'], fields=['text'], term_statistics=True, positions=False)
    if 'text' in tv['term_vectors']:   # just in case some document has no field named 'text'
        for t in tv['term_vectors']['text']['terms']:
            word = t
            count = tv['term_vectors']['text']['terms'][t]['term_freq']
            df = tv['term_vectors']['text']['terms'][t]['doc_freq']
            #pprint(tv['term_vectors']['text']['terms'][t])
            word_counts.update({word: count})
            doc_counts.update({word: df})

In [27]:
# show word frequencies
word_counts.most_common()

[('aliquam', 20),
 ('consectetur', 20),
 ('magnam', 18),
 ('amet', 17),
 ('quaerat', 17),
 ('est', 16),
 ('velit', 16),
 ('dolore', 15),
 ('voluptatem', 15),
 ('non', 14),
 ('quisquam', 14),
 ('dolorem', 13),
 ('ipsum', 13),
 ('labore', 13),
 ('neque', 13),
 ('numquam', 13),
 ('quiquia', 13),
 ('tempora', 12),
 ('ut', 12),
 ('dolor', 12),
 ('eius', 11),
 ('sit', 11),
 ('adipisci', 10),
 ('etincidunt', 9),
 ('porro', 9),
 ('sed', 8),
 ('modi', 4)]

In [28]:
# show doc freq
doc_counts.most_common()

[('aliquam', 10),
 ('labore', 10),
 ('magnam', 10),
 ('adipisci', 9),
 ('amet', 9),
 ('dolore', 9),
 ('est', 9),
 ('non', 9),
 ('tempora', 9),
 ('ut', 9),
 ('velit', 9),
 ('dolorem', 8),
 ('eius', 8),
 ('ipsum', 8),
 ('neque', 8),
 ('quaerat', 8),
 ('quisquam', 8),
 ('consectetur', 7),
 ('dolor', 7),
 ('porro', 7),
 ('sed', 7),
 ('sit', 7),
 ('voluptatem', 7),
 ('quiquia', 6)]

## 5. Proposed simple exercise

To get more familiar with elasticsearch, we propose that you _generate the Boolean and tf-idf matrices_ for the toy example that we used in class. You will find 7 text documents that contain the toy documents with the materials for this session in the racó. The steps to follow are:

- create an empty index
- open each text document in the `toy-docs` folder provided, read its contents and add it to the index as a new document; your index should contain 7 documents after this
- use the `termvectors` function to obtain term and doc counts, generate Boolean and tf-idf matrices based on these counts
- double check that your results coincide with the numbers in theory slides

## 6. Cleanup

Finally, we remove the test index..

In [29]:
index.delete()

ObjectApiResponse({'acknowledged': True})